In [1]:
import os 
import numpy as np

import sys
print(sys.path)

from deephive.environment.deephive_utils import *
from deephive.environment.utils import *

['/Users/elotech/Desktop/DeepHiveV2/notebooks', '/opt/homebrew/Cellar/python@3.11/3.11.7/Frameworks/Python.framework/Versions/3.11/lib/python311.zip', '/opt/homebrew/Cellar/python@3.11/3.11.7/Frameworks/Python.framework/Versions/3.11/lib/python3.11', '/opt/homebrew/Cellar/python@3.11/3.11.7/Frameworks/Python.framework/Versions/3.11/lib/python3.11/lib-dynload', '', '/Users/elotech/.virtualenvs/mlEnv/lib/python3.11/site-packages', '/Users/elotech/Documents/CODES/array-scaler', '/Users/elotech/Desktop/DeepHiveV2/src', '/opt/homebrew/opt/python-tk@3.11/libexec']
Using device: cpu


In [2]:
# config_path = "config/config.json"
# # load config
# config = parse_config(config_path)

compare_num = 1
log_scale = True
all_symbols = ["-", "--", "-", "--", "-", "--"]
all_colors = ["r", "g", "b", "k", "m", "y"]



# load the .npy file in each experiment folder in the result_path

def compare_experiments(exp_list, result_path, titles, log_scale=False, compare_num=1):
    save_path = "/Users/elotech/Desktop/DeepHiveV2/experiments/results/comparisons/comparison_" + str(compare_num) +".png"

    symbol_list = []
    color_list = []
    label_list = []
    gbest_values = []
    for i, exp_num in enumerate(exp_list):
        exp_path = result_path + "experiment_" + str(exp_num) + "/"
        # load the .npy file
        gbest_values.append(np.load(exp_path + f"experiment_{exp_num}.npy") * -1)
        # get the label
        label_list.append(titles[i])
        # get the symbol
        symbol_list.append(all_symbols[i])
        # get the color
        color_list.append(all_colors[i])

    # plot the gbest_values
    plot_num_function_evaluation(fopt=np.array(gbest_values), label_list=label_list, symbol_list=symbol_list,
                                 color_list=color_list, save_dir=save_path,
                                 n_agents=10, opt_value=None, log_scale=log_scale)
    
    
# compare_experiments(exp_list, result_path, compare_num, titles, log_scale=log_scale)

In [40]:
result_path = "/Users/elotech/Desktop/DeepHiveV2/experiments/results/"
exp_list = [2000, 2001, 2002, 2003, 2004, 2005]
compare_num = 5
log_scale = False
#compare_experiments(exp_list=exp_list, result_path=result_path, titles=titles, log_scale=log_scale, compare_num=compare_num)

In [41]:
def load_json(config_path):
    with open(config_path, 'r') as f:
        config = json.load(f)
    return config

def mean_confidence_interval(data, confidence=0.95):
    # check if data is just a single array and reshape it to a 2D array
    if len(data.shape) == 1:
        data = data.reshape(1, -1)
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a, axis = 0), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

In [56]:
label_list = []
gbest_values = []
for i, exp_num in enumerate(exp_list):
    exp_path = result_path + "exp_" + str(exp_num) + "/"
    # load the .npy file
    gbest_values.append(np.load(exp_path + f"gbestVals.npy") * -1)
    # get the label
    summary_json = load_json(exp_path + "run_summary.json")
    title = summary_json["title"]
    label_list.append(title)

In [57]:
result_comparision = []
for i, gbest in enumerate(gbest_values):
    mean, lower, upper = mean_confidence_interval(gbest)
    result_comparision.append({
        "title": label_list[i],
        "mean": mean[-1],
        "lower": lower[-1],
        "upper": upper[-1],
    })
    #result_comparision[label_list[i]] = {"mean": mean[-1], "lower": lower[-1], "upper": upper[-1]}
    
    

In [58]:
result_comparision

[{'title': 'CEC17 Function 1 - NO SPLITTING-UNFREEZE',
  'mean': 203.9946956178679,
  'lower': 203.26454838310175,
  'upper': 204.72484285263405},
 {'title': 'CEC17 Function 1 - SPLITTING-WITH STDs',
  'mean': 201.76273159486092,
  'lower': 201.23864784999557,
  'upper': 202.28681533972627},
 {'title': 'CEC17 Function 1 - SPLITTING-WITH STDs - Dynamic-split',
  'mean': 201.99037841798642,
  'lower': 201.7030477597986,
  'upper': 202.27770907617423},
 {'title': 'CEC17 Function 1 - SPLITTING-TWO POLICIES',
  'mean': 201.58192001175743,
  'lower': 200.75692039171477,
  'upper': 202.4069196318001},
 {'title': 'CEC17 Function 1 - SPLITTING-TWO POLICIES - Dynamic-split',
  'mean': 201.67400865485885,
  'lower': 201.24072814564937,
  'upper': 202.10728916406833},
 {'title': 'CEC17 Function 1 - PSO',
  'mean': 205.6213291487532,
  'lower': 203.22781225608102,
  'upper': 208.0148460414254}]

In [59]:
# create a dataframe from the result_comparision
import pandas as pd
df = pd.DataFrame(result_comparision)
df.to_csv("/Users/elotech/Desktop/DeepHiveV2/experiments/results/comparison_" + str(compare_num) +".csv")

In [60]:
df

,title,mean,lower,upper
0,CEC17 Function 1 - NO SPLITTING-UNFREEZE,203.994696,203.264548,204.724843
1,CEC17 Function 1 - SPLITTING-WITH STDs,201.762732,201.238648,202.286815
2,CEC17 Function 1 - SPLITTING-WITH STDs - Dynam...,201.990378,201.703048,202.277709
3,CEC17 Function 1 - SPLITTING-TWO POLICIES,201.581920,200.756920,202.406920
4,CEC17 Function 1 - SPLITTING-TWO POLICIES - Dy...,201.674009,201.240728,202.107289
5,CEC17 Function 1 - PSO,205.621329,203.227812,208.014846


In [22]:
from deephive.environment.optimization_functions.cec2017 import functions

In [23]:
f1 = functions.all_functions[0]

In [35]:
params = np.array([[-55.28, -70.43]])
f_x = f1(params)
print(f_x)

[-133.77129983]


In [37]:
ScalingHelper.scale(params, d_min=-100, d_max=100)

array([[0.2236, 0.1478]])